In [1]:
# get environment variables
import os
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [5]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt

hr_grd_path = os.path.join(os.environ.get("prodir"),'waom2_grd.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(os.environ.get("prodir"),'waom5_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

lr_tds_path =  os.path.join(os.environ.get("rawdir"),'gdata','waom5_data','waom5_tds.nc')
lr_tds = xr.open_dataset(lr_tds_path)

hr_tds_path =  os.path.join(os.environ.get("rawdir"),'gdata','waom2_data','waom2_tds.nc')

In [11]:
hr_tds = lr_tds.copy()
hr_tds.tide_Pamp[0]

<xarray.DataArray 'tide_Pamp' (eta_rho: 1460, xi_rho: 1720)>
[2511200 values with dtype=float64]
Coordinates:
    tide_period  timedelta64[ns] 12:25:14.163600
Dimensions without coordinates: eta_rho, xi_rho

In [20]:
gt='rho'
((hr_grd['x_rho'].values == lr_grd['x_'+gt][-1,-1].values) & (hr_grd['y_'+gt].values == lr_grd['y_'+gt][-1,-1].values)).any()

True

In [ ]:
(hr_grd['x_'+gt].values == lr_grd['x_'+gt][0,0].values) & (hr_grd['y_'+gt].values == lr_grd['y_'+gt][0,0].values

In [12]:
test = low_to_high(hr_tds.tide_Pamp[0],lr_grd,hr_grd,'rho',2)

set up empty hr data array
find index of shared first coordinate
Fill in the mask of lr data


TypeError: only integer scalar arrays can be converted to a scalar index

In [13]:
%debug

> /home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/xarray/core/indexing.py(305)as_integer_or_none()
    303 
    304 def as_integer_or_none(value):
--> 305     return None if value is None else operator.index(value)
    306 
    307 

ipdb> u
> /home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/xarray/core/indexing.py(309)as_integer_slice()
    307 
    308 def as_integer_slice(value):
--> 309     start = as_integer_or_none(value.start)
    310     stop = as_integer_or_none(value.stop)
    311     step = as_integer_or_none(value.step)

ipdb> u
> /home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/xarray/core/indexing.py(331)__init__()
    329                 k = int(k)
    330             elif isinstance(k, slice):
--> 331                 k = as_integer_slice(k)
    332             else:
    333                 raise TypeError('unexpected indexer type for {}: {!r}'

ipdb> u
> /home/ubuntu/bigStick/anac

In [7]:
from scipy import interpolate

def low_to_high(lr_da,lr_grd,hr_grd,gt,dim,fill_value=0.0):
    
    print('set up empty hr data array')
    if dim == 2:
    
        dummy = np.zeros(hr_grd['lon_'+gt].shape)
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        hr_da = xr.DataArray(dummy,coords=[y,x],dims=['eta_'+gt,'xi_'+gt])
        
    elif dim == 3:
        
        N = lr_da.s_rho.size
        dummy = np.tile(np.zeros(hr_grd['lon_'+gt].shape),(N,1,1))
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        z = lr_da['s_rho']
        hr_da = xr.DataArray(dummy,coords=[z,y,x],dims=['s_rho','eta_'+gt,'xi_'+gt])
    
    print('find index of shared first coordinate')
    # Find index of bottom left corner of low res data on high res grid
    ind = (hr_grd['x_'+gt].values == lr_grd['x_'+gt][0,0].values) & (hr_grd['y_'+gt].values == lr_grd['y_'+gt][0,0].values)
    eta0, xi0 = np.array(np.nonzero(ind)).squeeze()
    
    print('Fill in the mask of lr data')
    # Fill the mask of low resolution data with nearest neibghours and fill in known values on high res grid.
    if dim == 2:
        data = lr_da.values

        valid_mask = ~np.isnan(data)
        coords = np.array(np.nonzero(valid_mask)).T
        values = data[valid_mask]

        it = interpolate.NearestNDInterpolator(coords,values)

        filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
        
        # Fill in known values on high res grid
        hr_da[eta0::2,xi0::2] = filled
        
    if dim == 3:
        
        for k in np.arange(N):
            
            print('processing depth level: ',k)
            data = lr_da[k].values

            valid_mask = ~np.isnan(data)
            coords = np.array(np.nonzero(valid_mask)).T
            values = data[valid_mask]

            it = interpolate.NearestNDInterpolator(coords,values)

            filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
    
            # Fill in known values on high res grid
            hr_da[k,eta0::2,xi0::2] = filled
    
    # Now interpolate the intermediate grid points using x and y coordinates (in km from center point)
    print('interpolate at new intermediate cells on the hr grid')
    
    # Points we know the data
    x = hr_grd['x_'+gt][0,xi0::2].values
    y = hr_grd['y_'+gt][eta0::2,0].values
    
    # Define the target cells as meshgrid
    grid_x,grid_y = np.meshgrid(hr_grd['x_'+gt][0,:].values,hr_grd['y_'+gt][:,0].values)
    
    if dim == 2:
        # Data at these points
        values = hr_da[eta0::2,xi0::2].to_masked_array()

        # Define the interpolation function (fast regular grid interpolation)
        interp_func = interpolate.RegularGridInterpolator((y,x),values,bounds_error=False,fill_value=None)

        # Interpolate using linear interpolation
        interp = interp_func((grid_y[:,:],grid_x[:,:]))

        # Assign new data to data array
        hr_da[:,:] = interp
        
        # Fill with zeros where mask is present
        print('fill hr mask areas with fill value: ',fill_value)
        hr_da.values[hr_grd['mask_'+gt].values == 0] = fill_value
        
    if dim == 3:
        
        for k in np.arange(N):
            print('processing depth level: ',k)
            # Data at these points
            values = hr_da[k,eta0::2,xi0::2].to_masked_array()

            # Define the interpolation function (fast regular grid interpolation)
            interp_func = interpolate.RegularGridInterpolator((y,x),values,bounds_error=False,fill_value=None) 

            # Interpolate using linear interpolation
            interp = interp_func((grid_y[:,:],grid_x[:,:]))

            # Assign new data to data array
            hr_da[k,:,:] = interp

            # Fill with zeros where mask is present
            print('fill hr mask areas with fill value: ',fill_value)
            hr_da[k].values[hr_grd['mask_'+gt].values == 0] = fill_value
    
    return hr_da
